Document Data Gatherer 
==========================================

### Authors:
- Kannan T M


Introduction to SQLAlchemy
==========================

In [1]:
#postgres_db_str = (f'postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}')
sqllite_db_str='sqlite:///:memory:'
oracle_db_str=""
mongo_db_str=""

In [2]:
import sqlalchemy

# Connect to the database...
from sqlalchemy import create_engine
engine = create_engine(sqllite_db_str, echo=False)

### Creating Database-Backed Classes
---

The ORM utilizes classes inheriting from the SQLAlchemy base class. 

In [3]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

Classes have RDBMS table attributes:
- Columns have datatypes (Integer, Float, etc.)
- Columns have constraints (Primary Key, Foreign Key, etc.)

In [4]:
from sqlalchemy import Column, Integer, String
from flask import Flask, jsonify
from sqlalchemy import inspect
class DocumentDataSource(Base):
    __tablename__ = 'documentdatasource'
    __table_args__ = {'extend_existing': 'True'} 
    id = Column(Integer, primary_key=True)
    datasourceName = Column(String)
    templateID = Column(String)
    tableName = Column(String)
    templaterequiredFields = Column(String)
    
    
    def toDict(self):
        return { c.key: getattr(self, c.key) for c in inspect(self).mapper.column_attrs }
#Defines to_string() representation 
    def __repr__(self):
        return "<DocumentDataSource(datasourceName='%s', templateID='%s', tableName='%s , templaterequiredFields='%s)>" % (
                self.datasourceName, self.templateID, self.tableName ,self.templaterequiredFields)
     

In [5]:
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship
class Address(Base):
    __tablename__ = 'address'
    __table_args__ = {'extend_existing': 'True'} 
    id=Column(Integer, primary_key=True)
    customerid =  Column(Integer, ForeignKey('customer.customerid'))
    houseNumber = Column(String)
    street = Column(String)
    zipCode = Column(String)
    state = Column(String)
    addresstype=Column(String)
    active=Column(String)
    email_address = Column(String, nullable=False)
    customer = relationship("Customer", back_populates="address")
    def toDict(self):
        return { c.key: getattr(self, c.key) for c in inspect(self).mapper.column_attrs }

In [6]:
class Customer(Base):
    __tablename__ = 'customer'
    __table_args__ = {'extend_existing': 'True'} 
    customerid=Column(Integer, primary_key=True)
    #customerid=Column(Integer)
    firstName = Column(String)
    lastName = Column(String)
    gender = Column(String)
    DOB = Column(String)
    address=relationship("Address")
    def toDict(self):
        return { c.key: getattr(self, c.key) for c in inspect(self).mapper.column_attrs }

In [7]:
#Customer.address = relationship("Address", order_by=Address.customerid, back_populates="customer")

In [8]:
class Transaction(Base):
    __tablename__ = 'transaction'
    __table_args__ = {'extend_existing': 'True'} 
    transactionid= Column(Integer, primary_key=True)
    customerid = Column(Integer)
    transactiontype = Column(String)
    dateofthetransaction=Column(String)
    def toDict(self):
        return { c.key: getattr(self, c.key) for c in inspect(self).mapper.column_attrs }

In [9]:
Base.metadata.create_all(engine)

### Creating a Session

---

In [10]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

In [11]:
session.add_all([
        DocumentDataSource(datasourceName='sqlite', templateID='5001', tableName='Payment',templaterequiredFields='customerid:'),
        DocumentDataSource(datasourceName='sqlite', templateID='5001', tableName='Address',templaterequiredFields=''),
        DocumentDataSource(datasourceName='sqlite', templateID='5001', tableName='Person',templaterequiredFields=''),
        DocumentDataSource(datasourceName='sqlite', templateID='6007', tableName='Customer',templaterequiredFields='customerid:firstName:lastName'),
        DocumentDataSource(datasourceName='sqlite', templateID='6007', tableName='Address',templaterequiredFields='houseNumber:street:zipCode:state'),
        DocumentDataSource(datasourceName='postgres', templateID='5002', tableName='address',templaterequiredFields=''),
        DocumentDataSource(datasourceName='oracle', templateID='5005', tableName='Person',templaterequiredFields='')])

In [12]:
session.add_all([
        Address(customerid=3,houseNumber='223', street='3634 USAA BLV', zipCode='78240',state='TX',addresstype='communication',active='yes',email_address='yiuewo@gmail.com'),
        Address(customerid=6,houseNumber='32', street='Main St', zipCode='67893',state='IL',addresstype='communication',active='yes',email_address='IOO@gmail.com'),
        Address(customerid=2,houseNumber='1', street='3rd Cross St', zipCode='45647',state='MI',addresstype='communication',active='yes',email_address='DF@gmail.com'),
        Address(customerid=8,houseNumber='133', street='Rose Bud Dr', zipCode='13523',state='NY',addresstype='communication',active='yes',email_address='QWE@gmail.com'),
        Address(customerid=9,houseNumber='1223', street='Clean Mub pass', zipCode='23234',state='AR',addresstype='communication',active='yes',email_address='nice@gmail.com')])

In [13]:
# customerid = Column(Integer, primary_key=True)
#     firstName = Column(String)
#     lastName = Column(String)
#     gender = Column(String)
#     DOB = Column(String)

session.add_all([
        Customer(customerid=3,firstName='Kannan', lastName='Malaichamy', gender='male',DOB='07-July-1978'),
        Customer(customerid=6,firstName='Jim', lastName='Parson', gender='male',DOB='03-Jan-1982'),
        Customer(customerid=2,firstName='Rose', lastName='Sunday', gender='female',DOB='01-Feb-1992'),
        Customer(customerid=8,firstName='Hendry', lastName='Paul', gender='male',DOB='05-Dev-1998'),
        Customer(customerid=9,firstName='Jack', lastName='Master', gender='male',DOB='07-Jan-1984')])

Finally, the transaction can be commited. The remaining changes are flushed to the database.

In [14]:
session.commit()

In SQLAlchemy all database operations are performed within a transaction. 

For example we can edit and query for objects:

In [15]:
import json
templateID='6007'
tablefieldlist={}
for eachDataSource in session.query(DocumentDataSource).filter(DocumentDataSource.templateID==templateID,DocumentDataSource.datasourceName != 'null').all():
    jsonStr = json.dumps(eachDataSource.toDict())
    tablefieldlist[eachDataSource.tableName]=(eachDataSource.templaterequiredFields)
    print(jsonStr)

print(tablefieldlist)

{"id": 4, "datasourceName": "sqlite", "templateID": "6007", "tableName": "Customer", "templaterequiredFields": "customerid:firstName:lastName"}
{"id": 5, "datasourceName": "sqlite", "templateID": "6007", "tableName": "Address", "templaterequiredFields": "houseNumber:street:zipCode:state"}
{'Customer': 'customerid:firstName:lastName', 'Address': 'houseNumber:street:zipCode:state'}


We can do a rollback and our changes are reset.

In [16]:
#session.rollback()

Our objects are reset to their pervious states and our uncommited objects are removed.

### Querying
---

In [17]:
# for instance in session.query(DocumentDataSource).order_by(DocumentDataSource.id):
#     #print(instance)
#     jsonStr = json.dumps(instance.toDict())
#     print(jsonStr)

In [26]:
dictfilt = lambda x, y: dict([ (i,x[i]) for i in x if i in set(y) ])

In [49]:
customerfieldstr=tablefieldlist['Customer']
addressfieldstr=tablefieldlist['Address']
customerfieldlist=customerfieldstr.split(":")
addressfieldlist=addressfieldstr.split(":")
print(customerfieldlist)
finalJsonStr="{\"Customer\":"
for eachuser in session.query(Customer).filter(Customer.customerid==customerid).all():
    jsonStr = json.dumps(dictfilt(eachuser.toDict(), customerfieldlist))
    print(jsonStr)
    finalJsonStr=finalJsonStr+jsonStr+","
    jsonStr = json.dumps(dictfilt(eachuser.address[0].toDict(), addressfieldlist))
    finalJsonStr=finalJsonStr+"\"Address\":"+jsonStr + "}"
    print(finalJsonStr)

['customerid', 'firstName', 'lastName']
{"customerid": 3, "firstName": "Kannan", "lastName": "Malaichamy"}
{"Customer":{"customerid": 3, "firstName": "Kannan", "lastName": "Malaichamy"},"Address":{"houseNumber": "223", "street": "3634 USAA BLV", "zipCode": "78240", "state": "TX"}}


### Common Filtering Operators

Here’s an overview of some common operators used by the `filter()` fucntion:

#### equals:

In [47]:
for user in session.query(User).filter(User.name == 'ed'):
    print(user)

<User(name='ed', fullname='Ed Jones', password='f8s7ccs')>


#### not equals:

In [48]:
for user in session.query(User).filter(User.name != 'ed'):
    print(user)

<User(name='wendy', fullname='Wendy Williams', password='foobar')>
<User(name='mary', fullname='Mary Contrary', password='xxg527')>
<User(name='fred', fullname='Fred Flinstone', password='blah')>


#### LIKE:

In [49]:
for user in session.query(User).filter(User.name.like('%ed%')):
    print(user)

<User(name='ed', fullname='Ed Jones', password='f8s7ccs')>
<User(name='fred', fullname='Fred Flinstone', password='blah')>


#### IN:

In [50]:
for user in session.query(User).filter(User.name.in_(['ed', 'wendy', 'jack'])):
    print(user)

<User(name='ed', fullname='Ed Jones', password='f8s7ccs')>
<User(name='wendy', fullname='Wendy Williams', password='foobar')>


#### NOT IN:

In [51]:
for user in session.query(User).filter(~User.name.in_(['ed', 'wendy', 'jack'])):
    print(user)

<User(name='mary', fullname='Mary Contrary', password='xxg527')>
<User(name='fred', fullname='Fred Flinstone', password='blah')>


#### AND:

In [52]:
for user in session.query(User).filter(User.name == 'ed', User.fullname == 'Ed Jones'):
    print(user)

<User(name='ed', fullname='Ed Jones', password='f8s7ccs')>


#### OR:

In [53]:
from sqlalchemy import or_
for user in session.query(User).filter(or_(User.name == 'ed', User.name == 'wendy')):
    print(user)

<User(name='ed', fullname='Ed Jones', password='f8s7ccs')>
<User(name='wendy', fullname='Wendy Williams', password='foobar')>


### Relationships
---

Multiple classes can be created and linked together using class relations.

In [54]:
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship

class Address(Base):
    __tablename__ = 'addresses'
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey('users.id'))
    user = relationship("User", back_populates="addresses")
    
    def __repr__(self):
        return "<Address(email_address='%s')>" % self.email_address

Once primary and foreign keys are established, classes can be connected together via relationships.

In [55]:
User.addresses = relationship("Address", order_by=Address.id, back_populates="user")

Base.metadata.create_all(engine) # Flush schema changes to the DBMS.

In the above mappings:
- ```user.addresses``` maps to a user's list of address objects.
- ```address.user``` maps to an address's parent user object.

We create a "Jack" object and it initially has no addresses.

In [56]:
jack = User(name='jack', fullname='Jack Bean', password='gjffdd')
jack.addresses

[]

We then add some address objects.

In [57]:
jack.addresses = [Address(email_address='jack@google.com'), Address(email_address='j25@yahoo.com')]

In [1]:
jack.addresses[1]

NameError: name 'jack' is not defined

We see that the "Jack" object's first address's user parameter refers back to the "Jack" object.

In [59]:
jack.addresses[1].user

<User(name='jack', fullname='Jack Bean', password='gjffdd')>

When we add "Jack" to the session and commit, the two address objects of "Jack" are also committed in a cascading fashion.

In [60]:
session.add(jack)

session.commit()

jack = session.query(User).filter_by(name='jack').one()
jack.addresses

[<Address(email_address='jack@google.com')>,
 <Address(email_address='j25@yahoo.com')>]